In [1]:
import os
os.chdir("../..")

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from utils.dataset_loader import load_datasets
from utils.model_utils import initialize_model
from utils.train_utils import train_model
from utils.metrics import evaluate_model
from utils.visualization import plot_training, plot_confusion_matrix
import optuna
import os

C:\Users\Yash\Documents\Development\Projects\final_year\env\Lib\site-packages\albumentations\__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.23 (you have 1.4.22). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"[INFO] Using device: {device}")

[INFO] Using device: cuda


In [4]:
os.makedirs("outputs/models", exist_ok=True)
os.makedirs("outputs/plots", exist_ok=True)

In [5]:
print("[INFO] Loading datasets...")
train_loader, val_loader, test_loader = load_datasets(
    "wildfire_dataset_scaled", batch_size=32, augmentation="augmented"
)
print("[INFO] Datasets loaded successfully!")

[INFO] Loading datasets...
[INFO] Datasets loaded successfully!


In [6]:
def objective(trial):
    print(f"[DEBUG] Starting trial {trial.number}...")

    # Hyperparameter suggestions
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-3, log=True)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "SGD", "AdamW"])
    print(f"[DEBUG] Suggested hyperparameters: lr={lr}, weight_decay={weight_decay}, optimizer={optimizer_name}")

    # Initialize model
    print("[DEBUG] Initializing efficientnet_b0 model...")
    model = initialize_model("efficientnet_b0", num_classes=2, pretrained=True, freeze_all=False, unfreeze_last_n=2)
    model.to(device)

    # Optimizer setup
    print("[DEBUG] Setting up optimizer...")
    if optimizer_name == "Adam":
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif optimizer_name == "SGD":
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
    elif optimizer_name == "AdamW":
        optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

    # Scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", factor=0.1, patience=3, threshold=1e-4)

    # Loss function
    criterion = nn.CrossEntropyLoss()

    # Train the model
    print("[INFO] Starting model training...")
    save_path = "outputs/models/efficientnet_b0_trial_best.pth"
    history = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        device=device,
        save_path=save_path,
        early_stop_patience=8,
        monitor_metric="val_f1",
    )
    print("[INFO] Training completed!")

    # Return validation recall
    val_recall = history["val_f1"][-1]
    print(f"[INFO] Trial {trial.number} - Final Validation Recall: {val_recall:.4f}")
    return val_recall


In [ ]:
# Run Optuna
print("[INFO] Running hyperparameter optimization...")
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)
print(f"[INFO] Best parameters found: {study.best_params}")

[I 2024-12-18 20:23:33,474] A new study created in memory with name: no-name-007cfd37-4605-4ea8-b261-b0cc35612540


[INFO] Running hyperparameter optimization...
[DEBUG] Starting trial 0...
[DEBUG] Suggested hyperparameters: lr=1.4654646339457767e-05, weight_decay=0.0004988312945695164, optimizer=SGD
[DEBUG] Initializing efficientnet_b0 model...
[DEBUG] Setting up optimizer...
[INFO] Starting model training...

Starting training...



Epoch [1]: Train Loss: 0.7001, Train Acc: 0.5231 | Val Loss: 0.7015, Val Acc: 0.5423, Val Recall: 0.7602, Val F1: 0.6703
[INFO] Best model saved with val_f1: 0.6703
[INFO] Learning rate adjusted to: 0.000015


Epoch [2]: Train Loss: 0.6941, Train Acc: 0.5363 | Val Loss: 0.6923, Val Acc: 0.5522, Val Recall: 0.7358, Val F1: 0.6679
[INFO] No improvement in val_f1. Patience: 1/8
[INFO] Learning rate adjusted to: 0.000015


Epoch [3]: Train Loss: 0.6937, Train Acc: 0.5416 | Val Loss: 0.6839, Val Acc: 0.5423, Val Recall: 0.7398, Val F1: 0.6642
[INFO] No improvement in val_f1. Patience: 2/8
[INFO] Learning rate adjusted to: 0.000015


...

Epoch [14]: Train Loss: 0.1625, Train Acc: 0.9364 | Val Loss: 0.2845, Val Acc: 0.8881, Val Recall: 0.9309, Val F1: 0.9105
[INFO] No improvement in val_f1. Patience: 1/8
[INFO] Learning rate adjusted to: 0.001478


Epoch [15]: Train Loss: 0.1768, Train Acc: 0.9184 | Val Loss: 0.2539, Val Acc: 0.9104, Val Recall: 0.9472, Val F1: 0.9283
[INFO] Best model saved with val_f1: 0.9283
[INFO] Learning rate adjusted to: 0.001478


Epoch [16]: Train Loss: 0.1573, Train Acc: 0.9369 | Val Loss: 0.2551, Val Acc: 0.8930, Val Recall: 0.9228, Val F1: 0.9135
[INFO] No improvement in val_f1. Patience: 1/8
[INFO] Learning rate adjusted to: 0.001478


Epoch [17]: Train Loss: 0.1383, Train Acc: 0.9486 | Val Loss: 0.2549, Val Acc: 0.8930, Val Recall: 0.9309, Val F1: 0.9142
[INFO] No improvement in val_f1. Patience: 2/8
[INFO] Learning rate adjusted to: 0.001478


Epoch [18] - Training:  83%|█████████████████████████████████████████████████████████████████▌             | 49/59 [00:07<00:01,  6.84it/s]

In [ ]:
# Train Final Model
print("[INFO] Training final model with best hyperparameters...")
final_model = initialize_model("efficientnet_b0", num_classes=2, pretrained=True, freeze_all=False, unfreeze_last_n=2)
final_model.to(device)

In [ ]:
# Optimizer
optimizer = optim.Adam(final_model.parameters(), lr=study.best_params["lr"], weight_decay=study.best_params["weight_decay"])
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", factor=0.1, patience=3, threshold=1e-4)

In [ ]:
# Final training
final_save_path = "outputs/models/efficientnet_b0_final.pth"
history = train_model(
    model=final_model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=nn.CrossEntropyLoss(),
    optimizer=optimizer,
    scheduler=scheduler,
    device=device,
    save_path=final_save_path,
    early_stop_patience=8,
    monitor_metric="val_recall",
)

In [ ]:
# Evaluation and Visualization
print("[INFO] Evaluating final model...")
metrics = evaluate_model(final_model, test_loader, ["No Fire", "Fire"], device)
plot_training(history, "outputs/plots/efficientnet_b0_training_curve.png")
plot_confusion_matrix(metrics["confusion_matrix"], ["No Fire", "Fire"], "outputs/plots/efficientnet_b0_confusion_matrix.png")
print("[INFO] Final results saved successfully!")